In [ ]:
#@title **Connect** with $google$ drive!
try:
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)
    COLAB = True
    print("Note: using Google CoLab")
except:
    print("Note: not using Google CoLab")
    COLAB = False

In [ ]:
################################################################
### Clear cache. 
################################################################
!pip install -U --no-cache-dir gdown --pre

# Load **generator** model.

In [ ]:
#@title Generator - **model** - functions.
def conv_block(inputs, num_filters, batch_norm=True):
  ################################################################
  ### TODO.
  ################################################################
  x = Conv2D(num_filters, 3, padding='same')(inputs)
  if batch_norm:
    x = BatchNormalization()(x)
  x = Activation('relu')(x)
  x = Conv2D(num_filters, 3, padding='same')(x)
  if batch_norm:
    x = BatchNormalization()(x)
  x = Activation('relu')(x)
  return x

def encoder_block(inputs, num_filters):
  ################################################################
  ### TODO.
  ################################################################
  x = conv_block(inputs, num_filters)
  p = MaxPool2D((2, 2))(x)
  return x, p

def decoder_block(inputs, skip_features, num_filters):
  ################################################################
  ### TODO.
  ################################################################
  x = Conv2DTranspose(num_filters, (2, 2), strides=2, padding='same')(inputs)
  x = Concatenate()([x, skip_features])
  x = conv_block(x, num_filters)
  return x

def build_generator(input_shape, classes=1, factor=2):
  ################################################################
  ### TODO.
  ################################################################
  inputs = Input(input_shape)
  s1, p1 = encoder_block(inputs, int(64*factor))
  s2, p2 = encoder_block(p1, int(64*factor))
  s3, p3 = encoder_block(p2, int(128*factor))
  s4, p4 = encoder_block(p3, int(256*factor))
  s5, p5 = encoder_block(p4, int(512*factor))
  s6, p6 = encoder_block(p5, int(512*factor))
  s7, p7 = encoder_block(p6, int(512*factor))
  s8, p8 = encoder_block(p7, int(512*factor))
  b1 = conv_block(p8, int(512*factor))
  d1 = decoder_block(b1, s8, int(512*factor))
  d2 = decoder_block(d1, s7, int(512*factor))
  d3 = decoder_block(d2, s6, int(512*factor))
  d4 = decoder_block(d3, s5, int(512*factor))
  d5 = decoder_block(d4, s4, int(256*factor))
  d6 = decoder_block(d5, s3, int(128*factor))
  d7 = decoder_block(d6, s2, int(64*factor))
  d8 = decoder_block(d7, s1, int(64*factor))
  outputs = Conv2D(classes, (1, 1), padding='same', activation='sigmoid')(d8)
  model = Model(inputs, outputs, name='Generator')
  return model

In [ ]:
################################################################
### Old model. v.1.
################################################################
gen_data, checkpoint, index = 'https://drive.google.com/file/d/10MLZKnRSqK623xOy7Q50l8rzp1G-dFUL/view?usp=sharing', 'https://drive.google.com/file/d/10XcbbbHb9t3iiba58zakvuWXPoAe1e35/view?usp=sharing', 'https://drive.google.com/file/d/10g22fY4G6OPpOzkWvVTIIgUXsiWasoxT/view?usp=sharing'

In [ ]:
#@title Download **generator** weights.
import gdown

def download_checkpoint(url, filename='checkpoint'):
  id = url[32:-17]
  url = f'https://drive.google.com/u/1/uc?id={id}&export=download'
  output = f'{filename}'
  gdown.download(url, output)


download_checkpoint(checkpoint)
download_checkpoint(index, 'aiva.index')
download_checkpoint(gen_data, 'aiva.data-00000-of-00001')

In [ ]:
################################################################
### Load weights.
################################################################
generator.load_weights('aiva')

# Load **darknet** model.

In [ ]:
#@title Clone **darknet** repository. https://github.com/AlexeyAB/darknet { display-mode: "form" }
!sudo apt update
!git clone https://github.com/AlexeyAB/darknet
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!sed -i 's/CUDNN_HALF=0/CUDNN_HALF=1/' Makefile
!sed -i 's/LIBSO=0/LIBSO=1/' Makefile
!make
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1V3vsIaxAlGWvK4Aar9bAiK5U0QFttKwq' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1V3vsIaxAlGWvK4Aar9bAiK5U0QFttKwq" -O yolov4-csp.weights && rm -rf /tmp/cookies.txt

Hit:1 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Hit:2 http://archive.ubuntu.com/ubuntu focal InRelease
Hit:3 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Get:4 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Hit:5 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Get:7 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease [24.3 kB]
Get:8 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Hit:9 http://ppa.launchpad.net/ubuntugis/ppa/ubuntu focal InRelease
Get:10 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Get:11 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Get:12 http://archive.ubuntu.com/ubuntu focal-updates/universe amd64 Packages [1,323 kB]
Get:13 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 Packages [3,069 kB]
Get:14 h

In [ ]:
#@title Create **network** and install necassary dependencies. { display-mode: "form" }
from darknet import *
network, class_names, class_colors = load_network("cfg/yolov4-csp.cfg", "cfg/coco.data", "yolov4-csp.weights")
n_width = network_width(network)
n_height = network_height(network)
# ------------------------------------------------------------
# Install necessary dependencies.
# ------------------------------------------------------------
!pip install pafy
!pip install youtube_dl
!pip install -U kora
!python -m pip install git+https://github.com/ehsanbehdad/Pytube

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 52.2 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 KB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 56.0 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/ehsanbehdad/Pytube to /tmp/pip-req-build-cf1dnlpt
  Running command git clone --filter=blob:none --quiet https://github.com/ehsanbehdad/Pytube /tmp/pip-req-build-cf1dnlpt
  Resolved https://github.com/ehsanbehdad/Pytube to commit 2a2fa1d49bd11287548d95b68e56d378034727a0
  Preparing metadata (setup.py) ... done
  Created wheel for pytub

# **Video** output.

In [ ]:
from pytube import YouTube
from imutils.video import FileVideoStream
from imutils.video import FPS
from IPython.display import HTML
import matplotlib.pyplot as plt
import cv2
import numpy as np
import os


def progress(value, max=100,title=''):
  ## -----------------------------------------------------------
  ## TODO.
  ## -----------------------------------------------------------
  return HTML("""
        <label>{title}</label>
        <br>
        <progress
            value='{value}'
            max='{max}',
            style='width: 90%'>
            {value}
        </progress>
  """.format(value=value, max=max,title=title))


def prediction(frame):
  ## -----------------------------------------------------------
  ## Create empty image.
  ## -----------------------------------------------------------
  darknet_image = make_image(n_width, n_height, 3)
  ## -----------------------------------------------------------
  ## Convert color format.
  ## -----------------------------------------------------------
  img_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
  ## -----------------------------------------------------------
  ## Resize the frame to the network dimensions.
  ## -----------------------------------------------------------
  img_resized = cv2.resize(img_rgb, (n_width, n_height),interpolation=cv2.INTER_LINEAR)
  img_height, img_width, _ = frame.shape
  ## -----------------------------------------------------------
  ## Define the ratio between the output frame size and the 
  ## network dimensions. 
  ## -----------------------------------------------------------
  width_ratio = img_width/n_width
  height_ratio = img_height/n_height
  copy_image_from_bytes(darknet_image, img_resized.tobytes())
  # <->-<->-<->-<->-<->-<->-<->-<->-<->-<->-<->-<->-<->-<->-<->-
  detections = detect_image(network, class_names, darknet_image)
  free_image(darknet_image)
  # <->-<->-<->-<->-<->-<->-<->-<->-<->-<->-<->-<->-<->-<->-<->-
  return detections, width_ratio, height_ratio


def convert_bounding_box(bounding_box, width_ratio, height_ratio, padding=5):
  ## --------------------------------------------------------------
  ## TODO
  ## --------------------------------------------------------------
  left, top, right, bottom = bbox2points(bounding_box)
  left, top, right, bottom = int((left-padding) * width_ratio), int((top-padding) * height_ratio), int((right+padding) * width_ratio), int((bottom+padding) * height_ratio)
  return left, top, right, bottom


def detect(url, width=1280, height=720, max_frames=None, output_filepath='output.avi', padding=10, border_color=(119, 209, 242), side_by_side=False):
  ## --------------------------------------------------------------
  ## Download YT-Video.
  ## --------------------------------------------------------------
  video = YouTube(url)
  video.streams.filter(progressive=True, file_extension='mp4').order_by('resolution').desc().first().download('video', 'tmp.mp4')
  ## --------------------------------------------------------------
  ## Get video paramters.
  ## --------------------------------------------------------------
  fvs = FileVideoStream('video/tmp.mp4').start()
  framerate = fvs.stream.get(cv2.CAP_PROP_FPS)
  framecount = fvs.stream.get(cv2.CAP_PROP_FRAME_COUNT)
  if max_frames is not None:
    framecount = max_frames
  length = video.length
  ## --------------------------------------------------------------
  ## Start timer & create video-writer.
  ## -------------------------------------------------------------- 
  fps = FPS().start()
  fourcc = cv2.VideoWriter_fourcc(*'DIVX')
  if side_by_side:
    out = cv2.VideoWriter(output_filepath, fourcc, framerate, (width*2,  height))
  else:
    out = cv2.VideoWriter(output_filepath, fourcc, framerate, (width,  height))
  ## --------------------------------------------------------------
  ## Progress display.
  ## -------------------------------------------------------------- 
  frame_counter = 0
  video_progress = display(progress(frame_counter, framecount,'Initializing ...'), display_id=True)
  ## --------------------------------------------------------------
  ## Loop over all frames.
  ## --------------------------------------------------------------
  while fvs.more():
    frame = fvs.read()
    if frame is None:
      break
    ## ------------------------------------------------------------
    ## Resize frame to match output dimensions.
    ## ------------------------------------------------------------
    frame = cv2.resize(frame, (width, height), interpolation=cv2.INTER_LINEAR)
    original_frame = frame.copy()
    detections, width_ratio, height_ratio = prediction(frame)
    ## ------------------------------------------------------------
    ## TODO.
    ## ------------------------------------------------------------
    images = []
    coordinates = []
    for label, confidence, bounding_box in detections:
      if label not in ['car']:
        continue
      left, top, right, bottom = convert_bounding_box(bounding_box, width_ratio, height_ratio, padding)
      left, right = np.clip(left, 0, width-1), np.clip(right, 0, width-1)
      top, bottom = np.clip(top, 0, height-1), np.clip(bottom, 0, height-1)
      image = preprocess_section(frame, left, top, right, bottom)
      images.append(image)
      coordinates.append([left, top, right, bottom])
    ## ------------------------------------------------------------
    ## TODO.
    ## ------------------------------------------------------------
    images = np.asarray(images)
    if len(images) > 0:
      masks = generator(images, training=True).numpy()
      masks = masks * 255.0
      ## ------------------------------------------------------------
      ## TODO.
      ## ------------------------------------------------------------
      for i in range(len(masks)):
        left, top, right, bottom = coordinates[i]
        mask = masks[i]
        mask = cv2.resize(mask, (right-left, bottom-top))
        alpha_mask = mask[:, :, 3].copy()
        mask = cv2.cvtColor(mask, cv2.COLOR_BGRA2RGB)
        frame[top:bottom, left:right][alpha_mask>0.5] = mask[alpha_mask>0.5].astype(int)
        if border_color is not None:
          cv2.rectangle(frame, (left, top), (right, bottom), border_color, 2)
    ## ------------------------------------------------------------
    ## TODO.
    ## ------------------------------------------------------------
    if side_by_side:
      sbs_frame = np.zeros((height, width*2, 3))
      sbs_frame[:, :width] = frame
      sbs_frame[:, width:] = original_frame
      frame = sbs_frame
    out.write(frame.astype(np.uint8))
    video_progress.update(progress(frame_counter, framecount,'Progress: [' + str(frame_counter) + ' | ' + str(framecount) + '] - frames'))
    frame_counter += 1
    if max_frames is not None:
      if frame_counter >= max_frames:
        break

  fps.stop()
  out.release()
  cv2.destroyAllWindows()


def preprocess_section(frame, left, top, right, bottom):
  ## ------------------------------------------------------------
  ## TODO.
  ## ------------------------------------------------------------
  image = frame[top:bottom, left:right].copy()
  image = cv2.resize(image, (256, 256))
  image = image / 255.0
  return image


In [ ]:
detect('https://www.youtube.com/watch?v=-2RCPpdmSVg&ab_channel=PavelJi%C5%99%C3%ADk', 
       max_frames=None, output_filepath=os.path.join(os.pardir, 'tmp2.avi'),
       padding=3, side_by_side=True, border_color=None)